## Импорт 

In [ ]:
import pandas as pd
!pip3 install catboost
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
RANDOM_STATE=1234
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import numpy as np


## Разведочный анализ данных

### Общая информация, целевой признак.

- `age` - возраст пациента (0-120, старше и младше не бывает)
- `anaemia` - снижение показателей красных кровяных телец или гемоглобина (бинарный)
- `creatinine_phosphokinase` - уровень фермента КФК в крови (мкг/л)
- `diabetes` - наличие диабета (бинарный)
- `ejection_fraction` - процент крови, покидающей сердце при каждом сокращении (в процентах, диапазон значений от 0 до 100)
- `high_blood_pressure` - наличие гипертонии (бинарный)
- `platelets` - тромбоциты в крови (1000/мл)
- `serum_creatinine` - уровень сывороточного креатинина в крови (мг/дл)
- `serum_sodium` - уровень сывороточного натрия в крови (мэкв/л)
- `sex` - женщина или мужчина (бинарный)
- `smoking` - курит пациент или нет (бинарный)
- `time` - период наблюдения (дни) 
- `DEATH_EVENT` - целевой признак, пациент умер в течение периода наблюдения (бинарный)

In [ ]:
data.sample(8)

In [ ]:
data = data.rename(columns = {'DEATH_EVENT':'death_event'})

In [ ]:
data.info()

Всего в датасете 299 строк. Пропущенных значений нет. Признаки в основном целочисленные, три с плавающей запятой.

Преобразуем тип данных столбца age.

In [ ]:
data['age'] = data['age'].astype(int)

In [ ]:
data.duplicated().sum()

Дублей не наблюдается.


Посмотрим на описание данных. Проверим, что все признаки неотрицательные.

In [ ]:
data.describe()

Да, все неотрицательные, в допустимых диапазонах (для тех значений, которые мы обозначили ранее).


Посмотрим как выглядит целевой признак, оценим дисбаланс классов.

In [ ]:
#sns.countplot('death_event', data=data)
sns.countplot(x=data['death_event'])

Зафиксированых случаев смерти почти в два раза меньше чем случев с положительным исходом, таким образом имеется явный дисбаланс классов.

Посмотри еще раз на описание данных с целью понимания разброса значений признаков.

In [ ]:
data.describe().T

Итак, почти все признаки, кроме `creatinine_phosphokinase` и `serum_creatinine`, показывают нормальное распределение, поскольку их медиана и среднее значение(второй квартиль) почти совпадают. Для визуализации распределений воспользуемся функцией displot().

In [ ]:
list_of_numerical_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']

fig, ax = plt.subplots(nrows = 3,ncols = 2,figsize = (20,20))
for i in range(len(list_of_numerical_features)):
    plt.subplot(4,2,i+1)
    sns.distplot(data[list_of_numerical_features[i]],color = 'g', kde_kws = {'bw' : True});
    title = list_of_numerical_features[i]
    plt.title(title)

In [ ]:
#list_of_categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking', 'death_event']

#fig, ax = plt.subplots(nrows = 3,ncols = 2,figsize = (15,15))
#for i in range(len(list_of_categorical_features)):
    #plt.subplot(3,2,i+1)
    #sns.distplot(data[list_of_categorical_features[i]],color = 'g', kde_kws = {'bw' : True});
    #title = list_of_categorical_features[i]
    #plt.title(title)

Используя тепловую карту, посмторим на корреляцию величин.

In [ ]:
correlation_matrix = data.corr()

plt.figure(figsize= (16, 8))
sns.heatmap(correlation_matrix, annot = True)

Видим, что положительная корреляция есть между полом и курением. Отрицательная корреляция наблюдается между временем наблюдения и целевым признаком. Несильная связь есть между целевым признаком и возрастом, уровнем креатинина и натрия (разнонаправленные) и размером выброса крови при сокращении сердца. 

С помощью boxplot() удобно проверить наличие выбросов(для небинарных признаков), swarmplot() наглядно продемонстрирует распределение значений признаков, а так же их взаимосвязь с целевым признаком.

In [ ]:
feature_for_boxplot = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets','time','serum_creatinine', 'serum_sodium']
for i in range(len(feature_for_boxplot)):
    plt.figure(figsize=(10,10))
    #sns.stripplot(y=data[feature_for_boxplot[i]], x=data['death_event'], color='black')
    sns.swarmplot(y=data[feature_for_boxplot[i]], x=data['death_event'], color='black')
    sns.boxplot(y=data[feature_for_boxplot[i]], x=data['death_event'])
    plt.show


Почти во всех признаках наблюдаются выбросы. Учитывая размер и релевантость выборки выбросы удалять не будем.
Выводы: чем страше возраст, тем больше  смертей фиксируется, то есть прослеживается положительная корреляция. По мере уменьшения `ejection_fraction`  люди умирают больше, для `serum_sodium` аналогично. Для `creatinine_phosphokinase` медианы слева накладываются друг на друга - выводы относительно этого признака сделать нельзя на данном этапе, возможно, что имеется более сложная взаимосвязь между величиной этого параметра и смертностью, нечто похожее наблюдается и в случае признака `platelets` . При повышении `serum_creatinine`  наблюдается повышени смертности, то есть мы видим положительную корреляцию. Относительно времени можно заключить следующее: чем меньше время наблюдения, тем выше вероятность летального исхода, иными словами, чем больше человек под наблюдением, тем выше шанс выживания.

Проанализируем, как зависит целевой признак от наличия того или иного признака(бинарного).

In [ ]:
list_of_categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking', 'death_event']
for i in range(len(list_of_categorical_features)-1):
    table = data.pivot_table(index=list_of_categorical_features[i], values='death_event', aggfunc=['sum','count'])
    table.columns = ['number_of_death','number_of_observations']
    table['mortality_rate'] = table['number_of_death']/table['number_of_observations']
    display(table)
    sns.countplot(x = data[list_of_categorical_features[i]],data=data, hue='death_event');
    title = list_of_categorical_features[i] + ' and death_event' 
    plt.title(title)
    plt.show()


Согласно графикам и расчетам в таблицах можно отметить следующее: риски смертности увеличивается при наличии анемии и высокого давления, диабет, курение и пол не повышают риски.

Но, очень важно отметить, что имеется явный дисбаланс в данных, на последнем примере это очень хорошо видно, подгруппа курящих пациентов более чем в два раза меньше подгруппы некурящих, это затрудняет сделать правильный вывод относительно смертности по данному признаку. Это распространяется и на другие группы.

Поэтому, основываясь на данных, которые у нас имеются, мы не можем с уверенностью считать, что наши выводы до конца корректны. 

In [ ]:
list_drop = list_of_categorical_features + ['time'] 
list_drop.remove('death_event')
data_numerical = data.drop(list_drop, axis=1)
display(data_numerical)
sns.pairplot(data_numerical, hue='death_event')

Видим, что данные кучкуются в некоторых параметрах (хорошо видно на времени наблюдения, как на первых неделях высока смертность, на объёме сердцебиения можно заметить). По диагональным распределениям можно сказать, что чем сильнее различаются кривые, тем заметней влияние величин друг на друга. Скорее всего, влияние параметра, связанного с тромбоцитами, будет несущественное, его можно исключить из параметров для обучения модели предсказания. Оранжевая кривая полностью скрывается под синей, симметрично и с максимумом на одной оси (визуально).

### Моделирование

 В переменных features_train, target_train и features_test,target_test сохраним данные для тренировки и теста, соответственно. 

In [ ]:
features = data.drop(['death_event'], axis=1) # признаки
target = data['death_event'] # целевые признаки

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state = 12345, stratify=target)


В качестве метрики используем f1-score, чтобы балансировать ошибки первого и второго рода. 

In [ ]:
rating_list = {
    'f1-score': [],
    'recall-score': [],
    'precision-score': []
}

In [ ]:
def print_scores(model, features, target):
    score_f1 = cross_val_score(model, features, target, scoring="f1", cv=5)
    score_recall = cross_val_score(model, features, target, scoring="recall", cv=5)
    score_precision = cross_val_score(model, features, target, scoring="precision", cv=5)

    print('Диапазон значений метрик:')
    print('f1-score:', score_f1)
    print("recall-score:", score_recall)
    print("precision-score:", score_precision)
    print('\nСредние значения метрик:')
    print('f1-score:', score_f1.mean())
    print("recall-score:", score_recall.mean())
    print("precision-score:", score_precision.mean())
    #print(sum(score_recall)/len(score_recall))
    
    
    rating_list['f1-score'].append(score_f1.mean())
    rating_list['recall-score'].append(score_recall.mean())
    rating_list['precision-score'].append(score_precision.mean())
    
   

In [ ]:
# случайный лес
model_forest = RandomForestClassifier(random_state=RANDOM_STATE)
model_forest.fit(features_train, target_train)
# predicted = model.predict(features_test)
# print('f1-score:', f1_score(target_test, predicted))
# print("roc-auc-score:" , roc_auc_score(target_test, predicted))

print_scores(model_forest, features_train, target_train)


In [ ]:
# логистическая регрессия
model_log = LogisticRegression(random_state=RANDOM_STATE)
model_log.fit(features_train, target_train)

print_scores(model_log, features_train, target_train)



In [ ]:
# CatBoostClassifier
model_cat = CatBoostClassifier(random_state=RANDOM_STATE, silent=True)
model_cat.fit(features_train, target_train)

print_scores(model_cat, features_train, target_train)


In [ ]:
rating_table = {
    'Model': ['RandomForestClassifier', 'LogisticRegression', 'CatBoostClassifier'],
    'f1-score': rating_list['f1-score'],
    'recall-score': rating_list['recall-score'],
    'precision-score': rating_list['precision-score']
}
rating_table = pd.DataFrame(rating_table)
rating_table.sort_values('f1-score', ascending=False).style.highlight_max(subset=['f1-score',\
                                                                    'recall-score', 'precision-score'])

Таким образом, случайный лес имеет лучшие оценки на валидационных данных по метрике f1.
При этом положительный класс лучше определяет CatBoost.

### Подбор параметров
Подберём параметры для каждой модели. Критерием будет метрика f1.

In [ ]:
# баланс классов
#target_train.value_counts(normalize=True)

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train) 
features_train_scaled = scaler.transform(features_train)

In [ ]:
rating_list = {
    'f1-score': [],
    'recall-score': [],
    'precision-score': []
}

#### случайный лес

In [ ]:
parameters = {'n_estimators' : range (1, 100, 2), 
              'max_depth' : range (1, 20, 2), 
              'bootstrap' : ('True', 'False'),
              'class_weight':('balanced', 'None','balanced_subsample', [0.30, 0.70], [0.70, 0.30])
             }

grid_forest = GridSearchCV(RandomForestClassifier(random_state=RANDOM_STATE), parameters,\
                           scoring='f1', cv = 5, n_jobs= -1)   # , class_weight = 'balanced'
grid_forest.fit(features_train_scaled, target_train)

grid_forest.best_params_

In [ ]:
model_forest = RandomForestClassifier(n_estimators=grid_forest.best_params_.get('n_estimators'),\
                                      max_depth=grid_forest.best_params_.get('max_depth'),\
                                      bootstrap=grid_forest.best_params_.get('bootstrap'), random_state=RANDOM_STATE, class_weight=grid_forest.best_params_.get('class_weight'))



In [ ]:
print('f1-score:', grid_forest.best_score_)

In [ ]:
score_recall = cross_val_score(model_forest, features, target, scoring="recall", cv=5)
score_precision = cross_val_score(model_forest, features, target, scoring="precision", cv=5)
rating_list['f1-score'].append(grid_forest.best_score_)
rating_list['recall-score'].append(score_recall.mean())
rating_list['precision-score'].append(score_precision.mean())


#### логистическая регрессия

In [ ]:
parameters = {'solver':('liblinear', 'newton-cg', 'sag', 'saga', 'lbfgs'), 
              'max_iter': range(1,120, 10),
              'class_weight':('balanced', 'None', [0.30, 0.70], [0.70, 0.30])
             }  

grid_log = GridSearchCV(LogisticRegression(random_state=RANDOM_STATE), parameters, scoring="f1",\
                        cv = 5)
grid_log.fit(features_train_scaled, target_train)

grid_log.best_params_

In [ ]:
model_log = LogisticRegression(solver=grid_log.best_params_.get('solver'),\
                               max_iter=grid_log.best_params_.get('max_iter'),\
                               class_weight=grid_log.best_params_.get('class_weight'), random_state=RANDOM_STATE)

#print_scores(model_log, features_train_scaled, target_train)

In [ ]:
print('f1-score:', grid_log.best_score_)

In [ ]:
score_recall = cross_val_score(model_log, features, target, scoring="recall", cv=5)
score_precision = cross_val_score(model_log, features, target, scoring="precision", cv=5)
rating_list['f1-score'].append(grid_log.best_score_)
rating_list['recall-score'].append(score_recall.mean())
rating_list['precision-score'].append(score_precision.mean())

#### CatBoost

In [ ]:
parameters_cat = {'l2_leaf_reg': range (1, 15, 2), 
                  'iterations' : range(1, 100, 10),
                  'learning_rate':[0.03, 0.07, 0.1, 0.13, 0.17],
                  'max_depth' : range(1, 10, 2),
                 }

grid_cat = GridSearchCV(CatBoostClassifier(random_state=RANDOM_STATE),\
                        parameters_cat, scoring="f1",  cv = 5, n_jobs= -1)
grid_cat.fit(features_train_scaled, target_train, silent=True)
grid_cat.best_params_

In [ ]:
model_cat = CatBoostClassifier(l2_leaf_reg=grid_cat.best_params_.get('l2_leaf_reg'),\
                               iterations=grid_cat.best_params_.get('iterations'),\
                               learning_rate=grid_cat.best_params_.get('learning_rate'),\
                               max_depth=grid_cat.best_params_.get('max_depth'),\
                               random_state=RANDOM_STATE, silent=True)

#print_scores(model_cat, features_train_scaled, target_train)

In [ ]:
print('f1-score:', grid_cat.best_score_)

In [ ]:
score_recall = cross_val_score(model_cat, features, target, scoring="recall", cv=5)
score_precision = cross_val_score(model_cat, features, target, scoring="precision", cv=5)
rating_list['f1-score'].append(grid_cat.best_score_)
rating_list['recall-score'].append(score_recall.mean())
rating_list['precision-score'].append(score_precision.mean())

In [ ]:
rating_table = {
    'Algorithm': ['RandomForestClassifier', 'LogisticRegression', 'CatBoostClassifier'],
    'f1-score': rating_list['f1-score'],
    'recall-score': rating_list['recall-score'],
    'precision-score': rating_list['precision-score']
}
rating_table = pd.DataFrame(rating_table)
rating_table.sort_values('f1-score', ascending=False).style.highlight_max(subset=['f1-score',\
                                                 'recall-score', 'precision-score'])

После масштабирования признаков и подбора параметров ситуация _________________

### Тестирование

In [ ]:
scaler = StandardScaler()
scaler.fit(features_test) 
features_test_scaled = scaler.transform(features_test)

#model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=120, class_weight='balanced')
best_model= model_log.fit(features_train_scaled, target_train)
predictions = best_model.predict(features_test_scaled)
#print("f1-score для логистической регрессии:", f1_score(target_test, predictions_test_LogisticRegression))
print('f1-score:', f1_score(target_test, predictions))
print('recall-score:', recall_score(target_test, predictions))
print("precision-score:" , precision_score(target_test, predictions))

In [ ]:
conf_matrix = confusion_matrix(target_test, predictions)
table_conf_matrix = pd.DataFrame(data=conf_matrix, columns=['prediction 0','prediction 1'],\
                                 index=['actual 0','actual 1'])
#labels = ['True Neg','False Pos','False Neg','True Pos']
#labels = np.asarray(labels).rashape(2,2)
plt.figure(figsize = (8,5))
sns.heatmap(table_conf_matrix, annot=True, cmap='Greens_r')

Оценка точности на тестовой выборке ____ по метрике f1 для модели _____________________.

### Выводы